## Import

In [1]:
from __future__ import print_function, division

import os
import numpy as np
import fitsio

import fsps
import sedpy
import prospect

from sedpy.observate import load_filters

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Setup the models

In [3]:
from prospect.models import model_setup
from prospect.sources import CSPBasis

In [4]:
#help(CSPBasis)

In [12]:
# Specify the run parameters.
run_params = {
    'param_file': '',
    'verbose': True,
    'debug': False,
              'outfile': 'redmapper-masses',
              # Fitter parameters
              'nwalkers': 128,
              'nburn': [32, 32, 64], 'niter': 512,
              'do_powell': True,
              'ftol': 0.5e-5, 'maxfev': 5000,
              'initial_disp': 0.1,
              # Data parameters
              #'snr': 20.0, # ???
              #'add_noise': False, # ???
              # Model parameters
              'mass': 1E10,
              'logzsol': -0.5,
              'tage': 12.0,
              'tau': 3.0,
              'dust2': 0.3,
              # Data manipulation parameters
              #'logify_spectrum': False,
              #'normalize_spectrum': False,
              #'wlo':3750., 'whi':7200.,
              # SPS parameters
              'zcontinuous': 1,
              }
print(run_params)

{'niter': 512, 'dust2': 0.3, 'verbose': True, 'tage': 12.0, 'logzsol': -0.5, 'do_powell': True, 'zcontinuous': 1, 'initial_disp': 0.1, 'tau': 3.0, 'outfile': 'redmapper-masses', 'nburn': [32, 32, 64], 'maxfev': 5000, 'debug': False, 'mass': 10000000000.0, 'nwalkers': 128, 'param_file': '', 'ftol': 5e-06}


In [13]:
sdss = ['sdss_{}0'.format(b) for b in ['u','g','r','i','z']]
wise = ['wise_{}'.format(b) for b in ['w1','w2']]
filternames = sdss + wise
print(filternames)

['sdss_u0', 'sdss_g0', 'sdss_r0', 'sdss_i0', 'sdss_z0', 'wise_w1', 'wise_w2']


In [7]:
# load sps model (default)
#sps = model_setup.load_sps(**run_params)
sps = CSPBasis(zcontinuous=1, compute_vega_mags=False)

In [11]:
filters = load_filters(filternames)
print(filters)
#help(filters[0])
#filters[0].load_filter
#filters[0].display()

[<class 'sedpy.observate.Filter'>(sdss_u0), <class 'sedpy.observate.Filter'>(sdss_g0), <class 'sedpy.observate.Filter'>(sdss_r0), <class 'sedpy.observate.Filter'>(sdss_i0), <class 'sedpy.observate.Filter'>(sdss_z0), <class 'sedpy.observate.Filter'>(wise_w1), <class 'sedpy.observate.Filter'>(wise_w2)]


In [7]:
# create a function to read in the redmapper catalog and extract relavent files
def readred():
    redfile = os.path.join(os.sep, 'global', 'work', 'projects', 
                           'redmapper', 'redmapper_isedfit_v5.10_centrals.fits.gz')
    info = fitsio.FITS(redfile)
    cat = info[1].read()
    
    return cat

In [14]:
# Build output dictionary which contains the input photometry (our version of load_obs).
def prospcat(redcat):
    mask = (redcat['IVARMAGGIES'] > 0) * 1
    # print(mask)
    obs = {}
    obs['maggies'] = redcat['MAGGIES']
    with np.errstate(divide='ignore'):
        obs['maggies_unc'] = 1.0/np.sqrt(redcat['IVARMAGGIES']) #[:3, :])
    obs['wavelength'] = None # not fitting spectra
    obs['filters'] = filternames
    obs['phot_mask'] = mask  # 1 = good, 0 = bad
    obs['isedfit_id'] = redcat['ISEDFIT_ID']
    
    return obs

In [9]:
redcat = readred()

In [10]:
redcat.dtype.names

('MEM_MATCH_ID',
 'Z',
 'RA',
 'DEC',
 'R',
 'P',
 'PFREE',
 'THETA_I',
 'THETA_R',
 'IMAG',
 'IMAG_ERR',
 'ZRED',
 'ZRED_E',
 'ZRED_CHISQ',
 'BCG_SPEC_Z',
 'Z_SPEC_INIT',
 'Z_INIT',
 'LAMBDA_CHISQ',
 'LAMBDA_CHISQ_E',
 'LAMBDA_ZRED',
 'LAMBDA_ZRED_E',
 'R_LAMBDA',
 'SCALEVAL',
 'MASKFRAC',
 'C_LAMBDA',
 'C_LAMBDA_ERR',
 'MAG_LAMBDA_ERR',
 'CHISQ',
 'Z_LAMBDA',
 'Z_LAMBDA_E',
 'EBV_MEAN',
 'LNLAMLIKE',
 'LNBCGLIKE',
 'LNLIKE',
 'PZBINS',
 'PZ',
 'NCROSS',
 'RMASK',
 'RA_ORIG',
 'DEC_ORIG',
 'W',
 'DLAMBDA_DZ',
 'DLAMBDA_DZ2',
 'DLAMBDAVAR_DZ',
 'DLAMBDAVAR_DZ2',
 'LAMBDA_CHISQ_C',
 'LAMBDA_CHISQ_CE',
 'NCENT',
 'NCENT_GOOD',
 'RA_CENT',
 'DEC_CENT',
 'ID_CENT',
 'LAMBDA_CHISQ_CENT',
 'ZLAMBDA_CENT',
 'P_BCG',
 'P_CEN',
 'Q_CEN',
 'P_FG',
 'Q_MISS',
 'P_SAT',
 'P_C',
 'BCG_ILUM',
 'ILUM',
 'Z_LAMBDA_RAW',
 'Z_LAMBDA_E_RAW',
 'LIM_EXPTIME',
 'LIM_LIMMAG',
 'LIM_LIMMAG_HARD',
 'KCORR_MSTAR',
 'KCORR_COEFFS',
 'KCORR_CHI2',
 'KCORR_UVFLUX',
 'FNUV_ABSMAG_00',
 'FNUV_ABSMAG_IVAR_00',
 'UGRI

In [15]:
obs = prospcat(redcat)

In [17]:
obs

{'filters': ['sdss_u0',
  'sdss_g0',
  'sdss_r0',
  'sdss_i0',
  'sdss_z0',
  'wise_w1',
  'wise_w2'],
 'isedfit_id': array([  168,   434,   666, ..., 34832, 34845, 34873], dtype=int32),
 'maggies': array([[  8.08168750e-08,   6.74991668e-07,   1.72655052e-06, ...,
           3.43672355e-06,   3.23942504e-06,   1.95509529e-06],
        [  1.02759463e-08,   5.21811359e-08,   1.95987283e-07, ...,
           3.73946961e-07,   5.84767633e-07,   3.87884427e-07],
        [  6.44870024e-09,   1.60980221e-07,   4.88213459e-07, ...,
           8.71438601e-07,   1.20944264e-06,   7.12299254e-07],
        ..., 
        [  6.34218580e-11,   1.55727442e-09,   6.92663615e-09, ...,
           1.99032932e-08,   4.07453697e-08,   2.86100370e-08],
        [  1.79813442e-09,   1.88698146e-09,   4.91593877e-09, ...,
           7.67191821e-09,   1.36742511e-08,   1.15819407e-08],
        [  1.94307570e-09,   2.22601315e-09,   9.05395225e-09, ...,
           2.15409379e-08,   2.61627928e-08,   1.76007653e-0

In [18]:
# Load the SPS object
def load_sps(zcontinuous=1, compute_vega_mags=False, **extras):
    sps = CSPBasis(zcontinuous=zcontinuous,
                   compute_vega_mags=compute_vega_mags)
    return sps